In [0]:
from pyspark.sql.functions import sum, count, when, col, desc, rank, asc

In [0]:
from pyspark.sql.window import Window

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
# Find race years for which data needs to be processed

race_results_list = spark.read.format('delta').load(f'{presentation_folder_path}/race_results')\
    .filter(f"file_date = '{v_file_date}'")

In [0]:
race_year_list = df_column_to_list(race_results_list, 'race_year')

In [0]:
race_results = spark.read.format('delta').load(f'{presentation_folder_path}/race_results')\
    .filter(col('race_year').isin(race_year_list))

In [0]:
constructor_standing_df = race_results\
    .groupBy('race_year', 'team')\
    .agg(sum('points').alias('total_points'), count(when(col('position') == 1, True)).alias('wins'))

In [0]:
constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))
final_df = constructor_standing_df.withColumn('rank', rank().over(constructor_rank_spec))

In [0]:
final_df.createOrReplaceTempView('v_constructor_standings')

In [0]:
%sql

SET hive.exec.dynamic.partition.mode=nonstrict;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_presentation.constructor_standings(
  race_year INT,
  team STRING,
  total_points INT,
  wins INT,
  rank INT
)
PARTITIONED BY(race_year)
--STORED AS PARQUET


In [0]:
%sql

-- INSERT OVERWRITE f1_presentation.constructor_standings
-- PARTITION (race_year)
-- SELECT 
--     team,
--     total_points,
--     wins,
--     rank,
--     race_year
-- FROM v_constructor_standings

In [0]:
%sql

MERGE INTO f1_presentation.constructor_standings tgt
USING v_constructor_standings tmp
ON tgt.team = tmp.team
  AND tgt.race_year = tmp.race_year
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *

In [0]:
%sql
select * from f1_presentation.constructor_standings
where race_year = 2021